# MLflow GenAI Evaluation for Vietnamese QA

This notebook demonstrates how to integrate MLflow GenAI evaluation framework with VNPT Track 2.

**Reference:** [MLflow GenAI Evaluation Quickstart](https://mlflow.org/docs/latest/genai/eval-monitor/quickstart/)

## Prerequisites

Before running this notebook, ensure:
1. MLflow server is running: `mlflow server`
2. Ollama service is running: `ollama serve`
3. Model is available: `ollama pull qwen3:1.7b`
4. Dependencies are installed: `uv sync --group development`

## Step 1: Import Libraries and Initialize Services

In [1]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import mlflow
from src.brain.inference.processor import QuestionProcessor
from src.brain.llm.services.ollama import OllamaService

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
import os

os.environ["MLFLOW_GENAI_EVAL_MAX_WORKERS"] = "10"

In [3]:
from mlflow.genai.datasets import create_dataset

In [4]:
mlflow.set_tracking_uri("http://localhost:5000")

In [5]:
import mlflow
from mlflow import MlflowClient
import shutil
import os
from mlflow.exceptions import RestException, MlflowException
from mlflow.entities.lifecycle_stage import LifecycleStage

def reset_mlflow_experiment_ULTIMATE(name: str):
    client = MlflowClient()

    # 1) Get experiment if it exists
    exp = client.get_experiment_by_name(name)

    if exp:
        exp_id = exp.experiment_id
        print(f"Found experiment {name} (ID: {exp_id}, Status: {exp.lifecycle_stage}) …")
        
        # --- Handle Deletion/Purge if Active ---
        if exp.lifecycle_stage == LifecycleStage.ACTIVE:
            print("Soft deleting active experiment...")
            try:
                client.delete_experiment(exp_id)
                print("Successfully soft-deleted experiment.")
            except RestException as e:
                if "RESOURCE_DOES_NOT_EXIST" in str(e):
                    print(f"Warning: ID {exp_id} not found on server during delete. Continuing.")
                else:
                    raise e
        elif exp.lifecycle_stage == LifecycleStage.DELETED:
            print("Experiment is already soft-deleted. Attempting to restore...")
            # If it's already soft-deleted, we need to restore it to active to use it, 
            # or permanently delete it to create a new one. We will RESTORE for reuse.
            try:
                client.restore_experiment(exp_id)
                print("Successfully restored soft-deleted experiment.")
            except RestException as e:
                # Catch case where it was deleted but not found/purged
                if "RESOURCE_DOES_NOT_EXIST" in str(e):
                    print(f"Warning: ID {exp_id} not found on server during restore. Continuing.")
                else:
                    raise e


        # Clean up local artifacts if they exist
        if exp.artifact_location.startswith("file://"):
            local_path = exp.artifact_location.replace("file://", "")
            if os.path.exists(local_path):
                print("Deleting local artifact files…")
                shutil.rmtree(local_path)
            else:
                print("No local artifacts found")
        else:
            print("Remote artifact store detected — no local cleanup performed.")
    else:
        print("Experiment not found — will create new")

    # 2) Set or Create a fresh experiment using name
    # mlflow.set_experiment(name) will now either:
    # A) Create a brand new experiment if it was permanently deleted or never existed.
    # B) Set the now 'active' (restored) experiment as active.
    # C) It might still fail if the original delete was a permanent server-side purge.
    
    # We use a try/except block here to catch the final set_experiment error
    try:
        mlflow.set_experiment(name) 
    except MlflowException as e:
        # This catches the error if set_experiment finds a DELETED one but can't restore it.
        # This is the last resort—if the experiment is already DELETED, the server may not allow restore
        # and we can't do a hard delete from the client if the server doesn't support it.
        if "Cannot set a deleted experiment" in str(e):
             print("\n🚨 Final step error: Experiment is soft-deleted and cannot be set active.")
             print("To fix this, you may need to manually delete the experiment permanently from the MLflow UI/Server.")
             return # Exit the function, as the reset failed.
        else:
            raise e

    new_exp = mlflow.get_experiment_by_name(name)
    print(f"\nNew active experiment: {new_exp.name} (ID: {new_exp.experiment_id}, Status: {new_exp.lifecycle_stage})")

# Call the ultimate robust function
# Note: Since the output says it was SOFT-DELETED, the next run will execute the 'elif exp.lifecycle_stage == LifecycleStage.DELETED:' block
reset_mlflow_experiment_ULTIMATE("VNPT hackathon")

Found experiment VNPT hackathon (ID: 1, Status: deleted) …
Experiment is already soft-deleted. Attempting to restore...
Successfully restored soft-deleted experiment.
Remote artifact store detected — no local cleanup performed.

New active experiment: VNPT hackathon (ID: 1, Status: active)


In [6]:
dataset = create_dataset(
    name="hackathon_eval_set",
    experiment_id=mlflow.get_experiment_by_name("VNPT hackathon").experiment_id,
    tags={"stage": "validation", "domain": "RAG"}
)

In [7]:
from src.brain.llm.services.ollama import OllamaService

In [8]:
llm = OllamaService(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
    model="qwen3:1.7b"
)

In [9]:
# test response
response = llm.generate(
    user_input="xin chào",
    stream=False
)
print(response)

<answer>Xin chào</answer>


In [10]:
import string
from ratelimit import limits, sleep_and_retry

def format_prompt(question, choices):
    labels = list(string.ascii_uppercase)

    choice_text = "\n".join(
        f"{labels[i]}) {choices[i]}"
        for i in range(len(choices))
    )

    prompt = f"""Câu hỏi: {question}
Các lựa chọn:
{choice_text}
Hãy chọn một đáp án đúng nhất ({", ".join(labels[:len(choices)])}). chỉ trả về 1 trong các lựa chọn như A, B, .v.v

Đáp án của bạn là:
    """
    return prompt

@sleep_and_retry
@limits(calls=10, period=60)  # 10 calls per minute
def rate_limited_predict_fn(*args, **kwargs):
  return qa_predict_fn(*args, **kwargs)

def qa_predict_fn(question: str, choices: list) -> str:
    prompt = format_prompt(question, choices)
    response = llm.generate(
        user_input=format_prompt(question, choices),
        stream=False
    )
    return response

In [11]:
import json

def load_eval_dataset(path):
    with open(path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

        eval_dataset = []
        for item in raw_data:
            answer_letter = item["answer"].strip()

            eval_dataset.append(
                {
                    "inputs": {
                        "question": item["question"],
                        "choices": item["choices"],
                    },
                    "expectations": {
                        "expected_response": item["answer"]
                    }
                }
            )
        return eval_dataset

In [12]:
path_evalset = "../../vnpt-track2/data/val.json"
eval_dataset = load_eval_dataset(path_evalset)
print(eval_dataset[:5])

[{'inputs': {'question': 'Đoạn thông tin:\n[1] Tiêu đề: Khỉ thí nghiệm\nNội dung: Khỉ thí nghiệm là thuật ngữ chỉ về các loài linh trưởng (trừ con người), thông thường là các loài khỉ được sử dụng trong thí nghiệm y khoa (NHPs). Khỉ bao gồm các loài khỉ thực sự và các loài khỉ lớn (Ape) được sử dụng cho các mục đích thí nghiệm khoa học, nhất là trong y học. Xuất phát từ sự tương đồng của về cấu trúc sinh học giữa các loài khỉ và người do đó chúng thường được nuôi để làm vật thí nghiệm liên quan đến y khoa, trên cơ sở đó sẽ nhân rộng ra cho con người.\nCó 22 loài khỉ đuôi dài, trong đó một số được sử dụng thường xuyên trong các thí nghiệm khoa học. Khỉ vàng là nguồn nguyên liệu đầu để sản xuất hàng chục triệu liều văcxin bại liệt mỗi năm, góp phần vào việc thanh toán hoàn toàn bệnh bại liệt tại Việt Nam vào những năm 2000 Được sử dụng trong sản xuất vacxin chống bệnh bại liệt trẻ em, làm vật mẫu, đối tượng nghiên cứu khoa học. Khỉ vàng được lựa chọn là đối tượng nghiên cứu của các đề tà

In [13]:
from mlflow.genai import scorer
from mlflow.genai.scorers import Correctness, Guidelines
import re

def extract_answer_regex(text):
    # Pattern: <answer> followed by (captured content), followed by </answer>
    pattern = r"<answer>(.*?)</answer>"
    
    # Search for the pattern in the text
    match = re.search(pattern, text)
    
    # If a match is found, return the content of the first capturing group (group 1)
    if match:
        return match.group(1)
    else:
        return None

@scorer
def extract_match(expectations, outputs) -> bool:
    """Evaluate if the answer is concise (less than 5 words)"""
    return expectations["expected_response"] == extract_answer_regex(outputs)


scorers = [
    extract_match
]

In [ ]:
import mlflow

results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=qa_predict_fn,
    scorers=scorers,
)

2025/12/13 00:47:53 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/12/13 00:47:53 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.


Evaluating:   0%|          | 0/93 [Elapsed: 00:00, Remaining: ?] 